In [ ]:
import json
from scripts import step_1_generate_roi, step_2_deploy, step_3_detect_candidates, step_4_query_patch, step_5_download_candidates

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
template = {
    "roi": {
        "name": "ROI Name",
        "region": False,
    },
    "dates": {
        "start": "2021-01-01",
        "end": "2022-07-01"
    },
    "data": {
        "mosaic_period": 3,
        "mosaic_method": "min",
        "spectrogram_interval": 2
    },
    "pixel": {
        "version": "0.0.11",
        "name": "spectrogram_v0.0.11_2021-07-13",
    },
    "patch": {
        "version": "weak_labels_3.0",
        "name": "weak_labels_3.0_batchnorm",
        "stride": 8,
    },
    "candidate_detect": {
        "pred_threshold": 0.3,
        "min_sigma": 5.0
    },
    "intersect": {
        "patch_threshold": 0.3,
    },
    "run_local": False
}

In [ ]:
country_list = ['Korea North', 'Korea South', 'New Caledonia']

In [ ]:
for country in country_list:
    config = template.copy()
    config['roi']['name'] = country.lower().replace(' ', '_')
    #config = json.dumps(config)
    with open(f"../pipeline/{country.lower().replace(' ', '_')}_config.json", 'w') as f:
        json.dump(config, f, indent=4)
    

In [ ]:
roi_failures = {}
for country in country_list:
    print(country)
    try:
        step_1_generate_roi.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        roi_failures[country] = e
        print(f"{country} failed: {e}")
        continue

In [ ]:
from scripts import step_2_deploy
detect_failures = {}
for country in [c for c in country_list if c not in roi_failures.keys() and c != 'Samoa']:
    print(country)
    try:
        step_2_deploy.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        detect_failures[country] = e
        print(f"{country} failed: {e}")
        continue

In [ ]:
candidate_failures = {}
for country in [c for c in country_list if c not in detect_failures.keys() and c not in roi_failures.keys()]:
    print(country)
    try:
        step_3_detect_candidates.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        candidate_failures[country] = e
        print(f"{country} failed: {e}")
        continue

# retry failed countries
for country in [c for c in candidate_failures.keys()]:
    print(country)
    try:
        step_3_detect_candidates.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
        del(candidate_failures[country])
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        candidate_failures[country] = e
        print(f"{country} failed: {e}")
        continue

In [ ]:
query_failures = {}
for country in country_list:
    print(country)
    config_path = f"../pipeline/{country.lower().replace(' ', '_')}_config.json"
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        # check if candidate detect product exists
        print(config['candidate_detect']['product_id'])
        step_4_query_patch.main(config_path=config_path)
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        query_failures[country] = e
        print(f"{country} failed: {e}")
        continue

In [ ]:
download_failures = {}
for country in country_list:
    print("Downloading", country)
    config_path = f"../pipeline/{country.lower().replace(' ', '_')}_config.json"
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        # check if candidate detect product exists
        product_id = config['intersect']['product_id']
        step_5_download_candidates.main(config_path=config_path)
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        download_failures[country] = e
        print(f"{country} failed: {e}")
        continue

In [ ]:
import os
import geopandas as gpd
import pandas as pd
combined_df = gpd.GeoDataFrame()
for f in os.listdir('../data/model_outputs/candidate_sites/0.0.11/'):
    if f.startswith('intersect_'):
        gdf = gpd.read_file(f'../data/model_outputs/candidate_sites/0.0.11/{f}')
        combined_df = gpd.GeoDataFrame(pd.concat([combined_df, gdf]))

run_title = 'caledonia_and_korea'
combined_df.to_file(f'../data/model_outputs/candidate_sites/0.0.11/{run_title}.geojson', driver='GeoJSON')

In [ ]:
import os
import geopandas as gpd
import pandas as pd
run_title = 'remainder_batch_run'

gdf = gpd.read_file(f'../data/model_outputs/candidate_sites/0.0.11/{run_title}.geojson')

In [ ]:
# remove duplicates from gdf
print(len(gdf))
gdf = gdf.drop_duplicates(subset=['id'])
print(len(gdf))
gdf.to_file(f'../data/model_outputs/candidate_sites/0.0.11/{run_title}.geojson', driver='GeoJSON')